# 1.1 RAG란 무엇인가?  
  
## 1.1.1 RAG란?  
검색 증강 생성(Retrieval-Augmented Generation)의 약자입니다.  
단순히 학습 데이터에 의존해 답변을 생성하는 것이 아닙니다.  
외부 데이터베이스나 문서에서 관련된 정보를 검색(Retrieval)후 생성(Generation)하는 기법을 의미합니다.  
  
## 1.1.2 RAG의 필요성  
기존에 학습된 데이터만을 사용하면 다음의 항목이 문제가 됩니다.  
* 최신 정보 반영 불가  
* 학습하지 않은 도메인에 대해서 사용불가  
* 환각현상 발생  
* 시간과 비용에 대한 투자  
  
## 1.1.3 RAG 핵심 원리  
핵심적인 3가지 단계로 구성됨.  
1) 검색  
- 사용자의 질문에 가장 관련성이 높은 정보를 검색하는 단계입니다.  
- 백터 유사도를 사용하는 경우가 많습니다.  
2) 정보 증강  
- 검색된 문서를 원래 입력에 추가해서 모델에 제공하는 단계입니다.  
3) 응답 생성  
- 기존의 LLM의 인퍼런스 단계와 동일합니다.  
  
## 1.1.4 RAG 구현 방법  
RAG를 구현하는 과정은 `DB구축 -> 백터검색 -> LLM 통합` 세단계로 나눌 수 있습니다.  
그래프 DB를 사용하는 방법도 있으나. 이 책에서는 백터만을 사용합니다.  
  
**(1) 데이터베이스 구축**  
RAG의 필수적인 초기 단계입니다.  
문서를 백터(Vector)로 변환하여 백터 데이터베이스에 저장하는 단계입니다.  
RAG에 사용할 수 있는 문서들은 다음과 같습니다.  
*\[RAG에 사용가능한 문서\]*  
* .json, .csv 등의 구조화된 데이터  
* .pdf, .docs등의 문서 파일  
* 데이터베이스(sql, nosql)  
* 웹사이트 크롤링 데이터  
LLM이 더 정확하고 신뢰성 있는 답변을 생성할 수 있도록 체계적이고 구조화된 방식으로 저장하는것이 핵심입니다.  
① openai의 `text-embedding-ada-002`같은 임배딩 모델을 사용하여 텍스트를 백터로 변환하고  
② FAISS, Pinecone, Weaviate, Chroma 같은 백터DB에 저장하여 사용합니다.  
  
**(2) 백터 검색**  
기존의 키워드 검색과는 다릅니다.  
의미적 유사성을 고려하여 정밀한 검색 결과를 제공할 수 있습니다.  

**(3) LLM과 통합**  
프롬프트에 넣는것과 다르지 않습니다.  
하지만, langchain 프레임웍은 이들을 잘 추상화 했습니다.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# PDF 로드
loader = PyPDFLoader("data/succulent.pdf")
documents = loader.load()

# 백터 DB 구성
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())

# LLM 모델 로드
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 검색을 위한 QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

# 쿼리
query = "우리회사의 최신 제품"
response = qa_chain.invoke({'query':query})
print(response.get('result'))